In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
def get_start_index(df, cnv_start):
    if cnv_start <= df.iloc[0].start:
        return 0
    else:
        start_index = df.index[df["start"] <= cnv_start].tolist()[-1]
        if df.iloc[start_index].end <= cnv_start:
            start_index += 1
        return start_index


def get_end_index(df, cnv_end):
    if cnv_end >= df.iloc[-1].end:
        return df.shape[0]-1
    else:
        end_index = df.index[df["end"] >= cnv_end].tolist()[0]
        if df.iloc[end_index].start >= cnv_end:
            end_index -= 1
        return end_index

In [ ]:
ratio_dict = {}
ratio_pkl_dir = "ABS_RATIO_DIR_FROM_STEP_5"
chr_list = list(map(str, range(1,23)))
for chrom in chr_list:
    pkl_path = os.path.join(ratio_pkl_dir, chrom+"_abslog2ratio.filtered.100.100.windows.pkl")
    df = pd.read_pickle(pkl_path)
    df.reset_index(drop=True)
    ratio_dict["chr"+chrom] = df.copy()

In [ ]:
raw_cnv = pd.read_pickle("OUTPUT_FROM_STEP_8")

In [ ]:
cnv_df = raw_cnv.copy()
cnv_df["N_window"] = 0
cnv_df["N_nonNA_window"] = 0

In [ ]:
for index, row in a_cnv.iterrows():
    ratio_df = ratio_dict[row["chrom"]]
    bc = row["barcode"]
    start_index = get_start_index(ratio_df, row["start"])
    end_index = get_end_index(ratio_df, row["end"])
    cell_ratio = ratio_df.loc[start_index:end_index].loc[:, bc]
    cnv_df.at[index, "N_window"] = len(cell_ratio)
    cnv_df.at[index, "N_nonNA_window"] = cell_ratio.notna().sum()

In [ ]:
cnv_df["nonNA_frac"] = cnv_df["N_nonNA_window"]/cnv_df["N_window"]

In [ ]:
cnv_df["nonNA_frac"].plot.hist(bins=100)

In [ ]:
#select cutoff from the distribution, we used 0.25 for our data
out_df = cnv_df.query("nonNA_frac >= 0.25").copy()

In [ ]:
out_cnv.to_csv("final_calls.csv", index=False, sep="\t")